In [ ]:
!pip -q install langchain huggingface_hub tiktoken
!pip -q install chromadb
!pip -q install PyPDF2 pypdf sentence_transformers
!pip -q install -U FlagEmbedding
!pip install InstructorEmbedding
!pip install accelerate bitsandbyte

* you might need to restart the kernel after installing accelerate


## Setting up LangChain


In [1]:
import os

In [91]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.llms import HuggingFacePipeline
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from transformers import pipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.chains import RetrievalQA
from langchain import PromptTemplate, LLMChain
from langchain.vectorstores import Chroma


## RetrievalQA with LLaMA 2-7B
* the model can be changed to any open source model on hugging face hub


# Note : changing the model will requrie changing the promot template

# Load the model


In [3]:
!huggingface-cli login  #sign in with your acceses token from HF to load meta mode (not requried if the model is open)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [92]:
# name of hugging face llm to be used
model_name = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch
from transformers.tools.agents import AutoModelForCausalLM
import os
from langchain.llms import HuggingFaceHub



tokenizer = AutoTokenizer.from_pretrained(model_name)
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit = True,
    device_map="auto")


task = "text-generation"
llm = pipeline( task='text-generation',
                           model=model,
                            tokenizer=tokenizer,
                            max_new_tokens=512)
llm = HuggingFacePipeline(pipeline=llm)



## Load multiple and process documents

In [4]:
# Load and process the text files
#change the path to your docs directory
docs_path='/content/new_papers/new_papers'
loader = DirectoryLoader(docs_path, glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [6]:
len(documents)

219

In [5]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
len(texts)

992

In [ ]:
texts[3]

Document(page_content='Contents\n1 Introduction 3\n2 Pretraining 5\n2.1 Pretraining Data . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\n2.2 Training Details . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\n2.3 Llama 2 Pretrained Model Evaluation . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7\n3 Fine-tuning 8\n3.1 Supervised Fine-Tuning (SFT) . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 9\n3.2 Reinforcement Learning with Human Feedback (RLHF) . . . . . . . . . . . . . . . . . . . . . 9\n3.3 System Message for Multi-Turn Consistency . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 16\n3.4 RLHF Results . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 17\n4 Safety 20\n4.1 Safety in Pretraining . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 20', metadata={'source': 'n

## HF Instructor Embeddings
# can choose any other open source embedding model :
 https://huggingface.co/spaces/mteb/leaderboard

In [6]:
#loading instruct model to use as embed model


from langchain.embeddings import HuggingFaceBgeEmbeddings


encode_kwargs = {'normalize_embeddings': True} # compute cosine similarity
embedding_model='hkunlp/instructor-large'
model_norm = HuggingFaceBgeEmbeddings(
    model_name=embedding_model,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)


## create the DB

 T4 GPU

In [7]:
%%time
# Getting the embeddings from the docs and storing them in a directory
# might take some time to process all the files
persist_directory = 'db'
embedding = model_norm
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

CPU times: user 1min 40s, sys: 220 ms, total: 1min 40s
Wall time: 1min 41s


##retriever

In [10]:
retriever = vectordb.as_retriever(search_kwargs={"k":5})   # k is the number of documents

In [21]:
## Default LLaMA-2 prompt style
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

In [22]:
#this promot with memory (chat -memory)
sys_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """CONTEXT:/n/n {context} Chat history:/n/n {history}

Question: {question}
Asisitant[/INST] """
get_prompt(instruction, sys_prompt)

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. \n<</SYS>>\n\nCONTEXT:/n/n {context} Chat history:/n/n {history}\n\nQuestion: {question}[/INST]"

In [94]:
# this template without chat memory (chat history),the model will not keep track of conversation but you can ask independant questions
template1=""" [INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
CONTEXT:/n/n {context}
Question: {question}
Asisitant[/INST] """

In [133]:
prompt1 = PromptTemplate(
    input_variables=["context", "question",'history'],
    template=template1
)

In [135]:
prompt1.template

" [INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. \n<</SYS>>\nCONTEXT:/n/n {context} \nQuestion: {question}\nAsisitant[/INST] "

## Creating a chain

In [41]:
from langchain.prompts import PromptTemplate
prompt_template = get_prompt(instruction, sys_prompt)

prompt2 = PromptTemplate(
    template=prompt_template, input_variables=["context", "question",'history']
)

In [24]:
prompt2

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. \n<</SYS>>\n\nCONTEXT:/n/n {context} Chat history:/n/n {history}\n\nQuestion: {question}[/INST]"

In [134]:
#from langchain.schema import prompt
# create the chain to answer questions
#you can change the here prompt without memory ,prompt without memory
memory = ConversationBufferMemory(
            memory_key="history",
            input_key="question",
            return_messages=False,

        )
chain_type_kwargs = {"verbose":False,"prompt":prompt1,"memory": memory }
print(chain_type_kwargs)


qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="stuff",
                                       verbose=False,
                                       retriever=retriever,
                                       chain_type_kwargs=chain_type_kwargs,
                                       return_source_documents=True)



{'verbose': False, 'prompt': PromptTemplate(input_variables=['context', 'question'], template=" [INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. \n<</SYS>>\nCONTEXT:/n/n {context} \nQuestion: {question}\nAsisitant[/INST] "), 'memory': ConversationBufferMemory(input_key='question')}


In [130]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [101]:
import torch ,gc
with torch.no_grad():
       gc.collect()
       torch.cuda.empty_cache()

In [131]:
# full example (the gpu migh overflow due to chat memory)
import torch ,gc
while True:
    query = input("Ask a question: ")
    if query.lower() == 'exit':

      break
    llm_response = qa_chain(query)
    process_llm_response(llm_response)
    with torch.no_grad():
         gc.collect()

Ask a question: exit


In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7e49506ea410>)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. 
<</SYS>>

CONTEXT:/n/n {context}/n

Question: {question}[/INST]
